In [1]:
# 경고 메시지 무시
import warnings
warnings.filterwarnings(action='ignore')

## Groupby

In [2]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

plt.rc('font', family='malgun gothic')    # 한글처리
plt.rcParams['axes.unicode_minus']=False  # '- 표시

In [3]:
mpg = pd.read_csv('Data/mpg.csv')
mpg.head()

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,category
0,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
1,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
2,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
3,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
4,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact


In [4]:
# manufacturer: 자동차 제조사 이름. "Toyota", "Ford", "Honda" 등
# model: 자동차 모델명. "Camry", "F-150", "Accord" 등
# displ: 엔진 배기량(displacement). 일반적으로 리터(liter) 단위로 표시.
# year: 자동차 생산 연도. 
# cyl: 엔진 실린더 수. 
# trans: 변속기 종류. 
# drv: 구동 방식. 전륜 구동(Front Wheel Drive), 후륜 구동(Rear Wheel Drive), 4륜 구동(Four Wheel Drive) 등
# cty: 도시 주행 연비(city miles per gallon).
# hwy: 고속도로 주행 연비(highway miles per gallon).
# fl: 자동차 연료 타입.
# category: 자동차의 크기 또는 유형을 나타내는 분류. "compact", "midsize", "suv" 등

In [14]:
# 제조회사별, 구동방식별, cty평균
mpg.groupby(['manufacturer','drv'])['cty'].mean()

manufacturer  drv
audi          4      16.818182
              f      18.857143
chevrolet     4      12.500000
              f      18.800000
              r      14.100000
dodge         4      12.000000
              f      15.818182
ford          4      13.307692
              r      14.750000
honda         f      24.444444
hyundai       f      18.642857
jeep          4      13.500000
land rover    4      11.500000
lincoln       r      11.333333
mercury       4      13.250000
nissan        4      13.750000
              f      20.000000
pontiac       f      17.000000
subaru        4      19.285714
toyota        4      14.933333
              f      21.368421
volkswagen    f      20.925926
Name: cty, dtype: float64

In [15]:
mpg.groupby(['manufacturer','drv']).agg(mean_cty = ('cty','mean'))

mean_cty
manufacturer drv           
audi         4    16.818182
             f    18.857143
chevrolet    4    12.500000
             f    18.800000
             r    14.100000
dodge        4    12.000000
             f    15.818182
ford         4    13.307692
             r    14.750000
honda        f    24.444444
hyundai      f    18.642857
jeep         4    13.500000
land rover   4    11.500000
lincoln      r    11.333333
mercury      4    13.250000
nissan       4    13.750000
             f    20.000000
pontiac      f    17.000000
subaru       4    19.285714
toyota       4    14.933333
             f    21.368421
volkswagen   f    20.925926

In [20]:
# Audi 의 구동방식별 빈도
mpg.query('manufacturer == "audi"').groupby(['drv']).agg(drv_count = ('drv','count'))

,drv_count
drv,
4,11
f,7


In [21]:
mpg.groupby(['drv']).agg(drv_count = ('drv','count'))

,drv_count
drv,
4,103
f,106
r,25


In [25]:
# suv 데이터 추출하여, Total 항목을 (= hwy+cty /2 ) 만들고,
# 제조사별 Total 평균을 내림차순읋 표시

mpg.query('category == "suv"').assign(total = (mpg['hwy'] +  mpg['cty'] )/2).groupby('manufacturer').agg(mean_tot = ('total', 'mean')).sort_values('mean_tot', ascending = False).head()


,mean_tot
manufacturer,
subaru,21.916667
toyota,16.312500
nissan,15.875000
mercury,15.625000
jeep,15.562500


In [27]:
mpg.query('category == "suv"')\
          .assign(total = (mpg['hwy'] +  mpg['cty'] )/2)\
          .groupby('manufacturer')\
          .agg(mean_tot = ('total', 'mean'))\
          .sort_values('mean_tot', ascending = False)\
          .head()


,mean_tot
manufacturer,
subaru,21.916667
toyota,16.312500
nissan,15.875000
mercury,15.625000
jeep,15.562500


## 데이터 합치기_ merge90, concat()

In [28]:
# 중간고사 데이터 만들기
test1 = pd.DataFrame({'id'      : [1, 2, 3, 4, 5],
                      'midterm' : [60, 80, 70, 90, 85]})

# 기말고사 데이터 만들기
test2 = pd.DataFrame({'id'    : [1, 2, 3, 4, 5],
                      'final' : [70, 83, 65, 95, 80]})

In [31]:
test1

,id,midterm
0,1,60
1,2,80
2,3,70
3,4,90
4,5,85


In [32]:
test2

,id,final
0,1,70
1,2,83
2,3,65
3,4,95
4,5,80


In [35]:
total = pd.merge(test1, test2, how = 'left' , on = 'id')
total

,id,midterm,final
0,1,60,70
1,2,80,83
2,3,70,65
3,4,90,95
4,5,85,80


In [38]:
t_name = pd.DataFrame({'nclass'  : [1, 2, 3, 4],
                     'teacher' : ['kim', 'lee', 'park', 'choi']})
t_name

,nclass,teacher
0,1,kim
1,2,lee
2,3,park
3,4,choi


In [37]:
exam = pd.read_csv("Data/exam.csv")
exam

,id,nclass,math,english,science
0,1,1,50,98,50
1,2,1,60,97,60
2,3,1,45,86,78
3,4,1,30,98,58
4,5,2,25,80,65
5,6,2,50,89,98
6,7,2,80,90,45
7,8,2,90,78,25
8,9,3,20,98,15
9,10,3,50,98,45


In [39]:
exam_new = pd.merge(exam, t_name, how = 'right' , on = 'nclass')
exam_new

,id,nclass,math,english,science,teacher
0,1,1,50,98,50,kim
1,2,1,60,97,60,kim
2,3,1,45,86,78,kim
3,4,1,30,98,58,kim
4,5,2,25,80,65,lee
5,6,2,50,89,98,lee
6,7,2,80,90,45,lee
7,8,2,90,78,25,lee
8,9,3,20,98,15,park
9,10,3,50,98,45,park


In [40]:
exam_new = pd.merge(exam, t_name, how = 'left' , on = 'nclass')
exam_new

,id,nclass,math,english,science,teacher
0,1,1,50,98,50,kim
1,2,1,60,97,60,kim
2,3,1,45,86,78,kim
3,4,1,30,98,58,kim
4,5,2,25,80,65,lee
5,6,2,50,89,98,lee
6,7,2,80,90,45,lee
7,8,2,90,78,25,lee
8,9,3,20,98,15,park
9,10,3,50,98,45,park


In [41]:
group_a = pd.DataFrame({'id'   : [1, 2, 3, 4, 5],
                        'test' : [60, 80, 70, 90, 85]})

group_b = pd.DataFrame({'id'   : [6, 7, 8, 9, 10],
                        'test' : [70, 83, 65, 95, 80]})

In [42]:
group_a

,id,test
0,1,60
1,2,80
2,3,70
3,4,90
4,5,85


In [43]:
group_b

,id,test
0,6,70
1,7,83
2,8,65
3,9,95
4,10,80


In [48]:
group_all = pd.concat([group_a, group_b], axis = 0)
group_all

,id,test
0,1,60
1,2,80
2,3,70
3,4,90
4,5,85
0,6,70
1,7,83
2,8,65
3,9,95
4,10,80


In [49]:
group_all2 = pd.concat([group_a, group_b], axis = 1)
group_all2

,id,test,id,test
0,1,60,6,70
1,2,80,7,83
2,3,70,8,65
3,4,90,9,95
4,5,85,10,80


In [50]:
df1 = pd.read_csv('Data/concat_1.csv')
df2 = pd.read_csv('Data/concat_2.csv')
df3 = pd.read_csv('Data/concat_3.csv')

In [51]:
df1

,A,B,C,D
0,a0,b0,c0,d0
1,a1,b1,c1,d1
2,a2,b2,c2,d2
3,a3,b3,c3,d3


In [52]:
df2

,A,B,C,D
0,a4,b4,c4,d4
1,a5,b5,c5,d5
2,a6,b6,c6,d6
3,a7,b7,c7,d7


In [54]:
df3

,A,B,C,D
0,a8,b8,c8,d8
1,a9,b9,c9,d9
2,a10,b10,c10,d10
3,a11,b11,c11,d11


In [55]:
print(df1.index)

RangeIndex(start=0, stop=4, step=1)


In [56]:
df1.columns

Index(['A', 'B', 'C', 'D'], dtype='object')

In [57]:
df1.values

array([['a0', 'b0', 'c0', 'd0'],
       ['a1', 'b1', 'c1', 'd1'],
       ['a2', 'b2', 'c2', 'd2'],
       ['a3', 'b3', 'c3', 'd3']], dtype=object)

In [59]:
r_concat = pd.concat([df1, df2, df3])
r_concat

,A,B,C,D
0,a0,b0,c0,d0
1,a1,b1,c1,d1
2,a2,b2,c2,d2
3,a3,b3,c3,d3
0,a4,b4,c4,d4
1,a5,b5,c5,d5
2,a6,b6,c6,d6
3,a7,b7,c7,d7
0,a8,b8,c8,d8
1,a9,b9,c9,d9


In [60]:
new_series = pd.Series(['n1', 'n2', 'n3', 'n4'])
new_series

0    n1
1    n2
2    n3
3    n4
dtype: object

In [61]:
pd.concat([df1,new_series])

,A,B,C,D,0
0,a0,b0,c0,d0,NaN
1,a1,b1,c1,d1,NaN
2,a2,b2,c2,d2,NaN
3,a3,b3,c3,d3,NaN
0,NaN,NaN,NaN,NaN,n1
1,NaN,NaN,NaN,NaN,n2
2,NaN,NaN,NaN,NaN,n3
3,NaN,NaN,NaN,NaN,n4


In [65]:
new_row_df = pd.DataFrame(
    data=[["n1", "n2", "n3", "n4"],["n5", "n6", "n7", "n8"]],
    columns=["A", "B", "C", "D"],
)
new_row_df

,A,B,C,D
0,n1,n2,n3,n4
1,n5,n6,n7,n8


In [66]:
pd.concat([df1,new_row_df])

,A,B,C,D
0,a0,b0,c0,d0
1,a1,b1,c1,d1
2,a2,b2,c2,d2
3,a3,b3,c3,d3
0,n1,n2,n3,n4
1,n5,n6,n7,n8


In [67]:
pd.concat([df1,new_row_df], ignore_index = True)

,A,B,C,D
0,a0,b0,c0,d0
1,a1,b1,c1,d1
2,a2,b2,c2,d2
3,a3,b3,c3,d3
4,n1,n2,n3,n4
5,n5,n6,n7,n8


In [68]:
col_concat = pd.concat([df1, df2, df3], axis = 'columns')
col_concat

,A,B,C,D,A,B,C,D,A,B,C,D
0,a0,b0,c0,d0,a4,b4,c4,d4,a8,b8,c8,d8
1,a1,b1,c1,d1,a5,b5,c5,d5,a9,b9,c9,d9
2,a2,b2,c2,d2,a6,b6,c6,d6,a10,b10,c10,d10
3,a3,b3,c3,d3,a7,b7,c7,d7,a11,b11,c11,d11


In [69]:
col_concat['A']

,A,A,A
0,a0,a4,a8
1,a1,a5,a9
2,a2,a6,a10
3,a3,a7,a11


In [70]:
col_concat['new_col'] = ['n1', 'n2', 'n3', 'n4',]
col_concat

,A,B,C,D,A,B,C,D,A,B,C,D,new_col
0,a0,b0,c0,d0,a4,b4,c4,d4,a8,b8,c8,d8,n1
1,a1,b1,c1,d1,a5,b5,c5,d5,a9,b9,c9,d9,n2
2,a2,b2,c2,d2,a6,b6,c6,d6,a10,b10,c10,d10,n3
3,a3,b3,c3,d3,a7,b7,c7,d7,a11,b11,c11,d11,n4


In [71]:
df1.columns = ['A', 'B', 'C', 'D']
df2.columns = ['E', 'F', 'G', 'H']
df3.columns = ['A', 'C', 'F', 'H']

In [72]:
df2

,E,F,G,H
0,a4,b4,c4,d4
1,a5,b5,c5,d5
2,a6,b6,c6,d6
3,a7,b7,c7,d7


In [73]:
row_con = pd.concat([df1, df2, df3])
row_con

,A,B,C,D,E,F,G,H
0,a0,b0,c0,d0,NaN,NaN,NaN,NaN
1,a1,b1,c1,d1,NaN,NaN,NaN,NaN
2,a2,b2,c2,d2,NaN,NaN,NaN,NaN
3,a3,b3,c3,d3,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,a4,b4,c4,d4
1,NaN,NaN,NaN,NaN,a5,b5,c5,d5
2,NaN,NaN,NaN,NaN,a6,b6,c6,d6
3,NaN,NaN,NaN,NaN,a7,b7,c7,d7
0,a8,NaN,b8,NaN,NaN,c8,NaN,d8
1,a9,NaN,b9,NaN,NaN,c9,NaN,d9


In [76]:
pd.concat([df1, df3], ignore_index = False, join = 'inner')
# 내부Join(공통항목 합치기), Default 는 outer (위의 Sample)

,A,C
0,a0,c0
1,a1,c1
2,a2,c2
3,a3,c3
0,a8,b8
1,a9,b9
2,a10,b10
3,a11,b11
